In [1]:
import sys
import pandas as pd
import numpy as np
import requests
import os
import datetime
from dateutil.relativedelta import relativedelta
import pytz
import matplotlib.dates as mdates
import json
import math
import ast
from sys import stdout as out
from fpdf import FPDF
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 14})
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [119]:
def get_dev_info(device, address):
    
    r = requests.post(address + "/api/auth/login",
                      json={'username': 'meazonpro@meazon.com', 'password': 'meazonpro1'}).json()
    
    # acc_token is the token to be used in the next request
    acc_token = 'Bearer' + ' ' + r['token']
    
    # get devid by serial name
    r1 = requests.get(
        url=address + "/api/tenant/devices?deviceName=" + device,
        headers={'Content-Type': 'application/json', 'Accept': '*/*', 'X-Authorization': acc_token}).json()
    
    label = r1['label']
    devid = r1['id']['id']
    r1 = requests.get(
        url=address + "/api/device/" + devid + "/credentials",
        headers={'Content-Type': 'application/json', 'Accept': '*/*', 'X-Authorization': acc_token}).json()
    devtoken = r1['credentialsId']

    r2 = requests.get(url=address + "/api/plugins/telemetry/DEVICE/"+devid+"/values/attributes",headers={'Content-Type': 'application/json', 'Accept': '*/*', 'X-Authorization': acc_token}).json()
    
    key_to_find = 'key'
    value_to_find1 = 'latitude'
    value_to_find2 = 'longitude'

    # Iterate through the list of dictionaries
    for dictionary in r2:
        if dictionary.get(key_to_find) == value_to_find1:
            lat = dictionary['value']
        if dictionary.get(key_to_find) == value_to_find2:
            lon = dictionary['value']
            
    
    # lat = r2[14]['value']
    # lon = r2[15]['value']
    return devid,devtoken,acc_token,label,lat,lon


def read_data(acc_token, devid, address, start_time, end_time, descriptors):

        
    r2 = requests.get(
        url=address + "/api/plugins/telemetry/DEVICE/" + devid + "/values/timeseries?keys=" + descriptors + "&startTs=" + start_time + "&endTs=" + end_time + "&agg=NONE&limit=1000000",
        headers={'Content-Type': 'application/json', 'Accept': '*/*', 'X-Authorization': acc_token}).json()
    
    if r2:
        df = pd.DataFrame([])
        
        for desc in r2.keys():
            df1 = pd.DataFrame(r2[desc])
            df1.set_index('ts', inplace=True)
            df1.columns = [str(desc)]
            
            df1.reset_index(drop=False, inplace=True)
            df1['ts'] = pd.to_datetime(df1['ts'], unit='ms')
            df1['ts'] = df1['ts'].dt.tz_localize('utc').dt.tz_convert('Europe/Athens')
            df1 = df1.sort_values(by=['ts'])
            df1.reset_index(drop=True, inplace=True)
            df1.set_index('ts', inplace=True, drop=True)            
            
            df = pd.concat([df, df1], axis=1)

        if df.empty:
            df = pd.DataFrame([])
        else:
            for col in df.columns:
                df[col] = df[col].astype('float64')
    else:
        df = pd.DataFrame([])
        # print('Empty json!')
    return df

In [236]:
address = 'https://mi6.meazon.com'
start_time = '1702634400000'
end_time = '1705312800000'


r = requests.post(address + "/api/auth/login",
                    json={'username': 'meazonpro@meazon.com', 'password': 'meazonpro1'}).json()

acc_token = 'Bearer' + ' ' + r['token']


entityId = '4795fc10-5b7f-11ee-b2c9-653b42f73605' # DEDDHE ATHINAS
r1 = requests.get(url=address + "/api/entityGroup/"+entityId+"/entities?pageSize=1000&page=0",headers={'Content-Type': 'application/json', 
'Accept': '*/*', 'X-Authorization': acc_token}).json()

In [104]:
# def find_alarms(acc_token, devid, start_time, end_time, syncal, devname):
#     alarmDesc = 'alarm_time,alarm_id,alarm_duration,alarm_value'
#     alarms = read_data(acc_token, devid, address,  start_time, end_time, alarmDesc)
#     if not alarms.empty:
#         alarms = alarms.sort_index()
#         alarms['alarm_id'] = alarms['alarm_id'].astype(int)
#         alarms['alarm_time'] = pd.to_datetime(alarms['alarm_time'], unit='ms')
#         alarms['alarm_time'] = alarms['alarm_time'].dt.tz_localize('utc').dt.tz_convert('Europe/Athens')
#         alarms = alarms.loc[alarms['alarm_id']==2]
#         if not alarms.empty:
#             alarms.rename(columns={'alarm_time':devname}, inplace=True)
#             syncal = pd.concat([syncal, alarms[devname]],axis=1)
#     return syncal

In [237]:
def find_alarms_dict(acc_token, devid, start_time, end_time, syncal, devname, assetname, lat, lon,label):
    alarmDesc = 'alarm_time,alarm_id,alarm_duration,alarm_value,alarm_status'
    alarms = read_data(acc_token, devid, address,  start_time, end_time, alarmDesc)
    if not alarms.empty:
        alarms = alarms.sort_index()
        alarms['alarm_id'] = alarms['alarm_id'].astype(int)
        alarms['alarm_time'] = pd.to_datetime(alarms['alarm_time'], unit='ms')
        alarms['alarm_time'] = alarms['alarm_time'].dt.tz_localize('utc').dt.tz_convert('Europe/Athens')
        alarms = alarms.loc[alarms['alarm_id']==2]
        if not alarms.empty:
            alarms.set_index('alarm_time', inplace=True,drop=False)
            alarms['device'] = devname
            alarms['line'] = assetname
            alarms['lat'] = lat
            alarms['lon'] = lon
            alarms['label'] = label
            # alarms.rename(columns={'alarm_time':'undervoltage_ts'}, inplace=True)
            alarms = alarms.resample('1T').first()
            alarms = alarms.dropna()
            # print(alarms)
            

            syncal = pd.concat([syncal, alarms[['device', 'label','line','alarm_duration','alarm_status','lat','lon','alarm_time']]],axis=0)
    return syncal

In [238]:
syncal = pd.DataFrame([])
for i in range(0,len(r1['data'])):
     #   os.chdir('/home/azureuser/deddhePDF/')
      assetid = r1['data'][i]['id']['id']
      assetname = r1['data'][i]['name']
      # print(assetname)
      if assetname[0]!='0':
         r2 = requests.get(url=address + "/api/relations/info?fromId="+assetid+"&fromType=ASSET",headers={'Content-Type': 'application/json', 'Accept': '*/*', 'X-Authorization': acc_token}).json()
         for j in range(0, len(r2)):
            device = r2[j]['toName']
            if device[:3]=='102':
               # print(device)
               [devid, _, acc_token,label, lat, lon] = get_dev_info(device, address)
               # print(label)
               # syncal = find_alarms(acc_token, devid, start_time, end_time,syncal, device)
               syncal = find_alarms_dict(acc_token, devid, start_time, end_time,syncal, device, assetname,lat,lon,label)

               # search for nested devices
               r3 = requests.get(url=address + "/api/relations/info?fromId="+devid+"&fromType=DEVICE",headers={'Content-Type': 'application/json', 'Accept': '*/*', 'X-Authorization': acc_token}).json()
               for k in range(0, len(r3)):
                  device = r3[k]['toName']
                  # print('nested device:', device)
                  [devid, _, acc_token,label, _, _] = get_dev_info(device, address)
                  # syncal = find_alarms(acc_token, devid, start_time, end_time,syncal, device)
                  syncal = find_alarms_dict(acc_token, devid, start_time, end_time,syncal, device, assetname,lat,lon,label)


In [239]:
syncal.head()

,device,label,line,alarm_duration,alarm_status,lat,lon,alarm_time
alarm_time,,,,,,,,
2024-01-07 01:25:00+02:00,102.408.000196,ΜΠ-013,ΜΠ-22 Κ/Δ ΠΑΛΛΗΝΗΣ,-1.0,3.0,37.957805,23.85225,2024-01-07 01:25:13+02:00
2024-01-07 01:44:00+02:00,102.408.000178,ΑΝΑΧΩΡΗΣΗ 6,ΜΠ-22 Κ/Δ ΠΑΛΛΗΝΗΣ,-1.0,3.0,37.957805,23.85225,2024-01-07 01:44:32+02:00
2023-12-19 07:44:00+02:00,102.408.000217,ΜΜ-014,1-16 ΜΑΡΚΟΠΟΥΛΟΥ,586.0,1.0,37.862998,23.85289,2023-12-19 07:44:44+02:00
2023-12-28 09:40:00+02:00,102.408.000217,ΜΜ-014,1-16 ΜΑΡΚΟΠΟΥΛΟΥ,579.0,1.0,37.862998,23.85289,2023-12-28 09:40:59+02:00
2024-01-01 10:32:00+02:00,102.408.000217,ΜΜ-014,1-16 ΜΑΡΚΟΠΟΥΛΟΥ,617.0,1.0,37.862998,23.85289,2024-01-01 10:32:12+02:00


In [240]:
# syncal = syncal.dropna(how='all')
# print(syncal.loc[syncal['alarm_duration']<0, ['alarm_status', 'alarm_duration']])
# syncal = syncal.loc[syncal['alarm_duration']>0]
syncal.loc[syncal['alarm_duration']<0,'alarm_duration']='Unknown'
syncal = syncal.drop('alarm_status',axis=1)

In [253]:
syncal.head()

,device,label,line,alarm_duration,lat,lon,alarm_ts
alarm_time,,,,,,,
2024-01-07 01:25:00+02:00,102.408.000196,ΜΠ-013,ΜΠ-22 Κ/Δ ΠΑΛΛΗΝΗΣ,Unknown,37.957805,23.85225,2024-01-07 01:25:13+02:00
2024-01-07 01:44:00+02:00,102.408.000178,ΑΝΑΧΩΡΗΣΗ 6,ΜΠ-22 Κ/Δ ΠΑΛΛΗΝΗΣ,Unknown,37.957805,23.85225,2024-01-07 01:44:32+02:00
2023-12-19 07:44:00+02:00,102.408.000217,ΜΜ-014,1-16 ΜΑΡΚΟΠΟΥΛΟΥ,586.0,37.862998,23.85289,2023-12-19 07:44:44+02:00
2023-12-28 09:40:00+02:00,102.408.000217,ΜΜ-014,1-16 ΜΑΡΚΟΠΟΥΛΟΥ,579.0,37.862998,23.85289,2023-12-28 09:40:59+02:00
2024-01-01 10:32:00+02:00,102.408.000217,ΜΜ-014,1-16 ΜΑΡΚΟΠΟΥΛΟΥ,617.0,37.862998,23.85289,2024-01-01 10:32:12+02:00


In [256]:
grouped = syncal.groupby('alarm_time')
i=0
with pd.ExcelWriter('syncalarms_with_ts.xlsx', engine='openpyxl') as writer:
    for name,group in grouped:
        
        if group.shape[0]>1:
            group = group.reset_index(drop=True)
            print(group)
            group['alarm_ts'] = group['alarm_ts'].astype(str)
            group.to_excel(writer, sheet_name='sheet_'+str(i),index=False)
            i+=1
    # print(group)

           device   label                  line alarm_duration        lat  \
0  102.408.000213  ΜΜ-218  3-22 Κ/Δ ΜΑΡΚΟΠΟΥΛΟΥ        Unknown  37.867379   
1  102.408.000214  ΜΜ-197  3-22 Κ/Δ ΜΑΡΚΟΠΟΥΛΟΥ        Unknown  37.862493   

         lon                  alarm_ts  
0  24.037221 2023-12-16 00:36:00+02:00  
1  24.034781 2023-12-16 00:36:00+02:00  
           device        label               line alarm_duration        lat  \
0  102.408.000228       ΜΓ-019  Ρ-230 ΝΕΑΣ ΜΑΚΡΗΣ          711.0  38.200179   
1  102.408.000220       ΜΤ-052  Ρ-230 ΝΕΑΣ ΜΑΚΡΗΣ          704.0  38.142461   
2  102.408.000017  ΑΝΑΧΩΡΗΣΗ 1  Ρ-230 ΝΕΑΣ ΜΑΚΡΗΣ          704.0  38.142461   
3  102.408.000192       ΜΝ-122  Ρ-210 ΝΕΑΣ ΜΑΚΡΗΣ          280.0  38.101213   
4  102.408.000211       ΜΤ-055  Ρ-210 ΝΕΑΣ ΜΑΚΡΗΣ           70.0  38.156282   

         lon                  alarm_ts  
0  23.997730 2023-12-16 01:42:54+02:00  
1  23.999470 2023-12-16 01:42:54+02:00  
2  23.999470 2023-12-16 01:42:51+02:00  
3  23.

In [245]:
syncal.rename(columns={'alarm_time':'alarm_ts'},inplace=True)

In [251]:
grouped = syncal.groupby('alarm_time')
# for name,group in grouped:
#     print(group)


In [124]:

# ## 1st try with geopandas
# for _, group in grouped:
    
#     geometry = [Point(lon, lat) for lon, lat in zip(group['lon'], group['lat'])]
#     gdf = gpd.GeoDataFrame(group, geometry=geometry, crs='EPSG:4326')
#     athens_shapefile = gpd.read_file('da_dim_koin.shp')
#     # Plotting using cartopy
#     fig, ax = plt.subplots(figsize=(10, 6))
#     # world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
#     athens_shapefile.plot(ax=ax, color='lightgray')

#     # Plot the points
#     gdf.plot(ax=ax, marker='o', color='red', markersize=50)

#     # Add labels
#     for x, y, label in zip(gdf['lon'], gdf['lat'], gdf['line']):
#         ax.text(x, y, label, fontsize=8, ha='right')

#     plt.title('Alarms in grid')
#     plt.show()

In [252]:
import folium
## 2nd try with shapely
i=0
for _, df in grouped:
    if df.shape[0]>1:
        
        # try:
            # define colors per group
        unlines = list(df['line'].unique())
        colors = ['red','blue','green','orange','purple','darkblue','gray','darkgreen']
        colordict = {}

        for m in range (0,len(unlines)):
            colordict[unlines[m]] = colors[m]
        
        colorlist = []
        for m in range(0, df.shape[0]):
            colorlist.append(colordict[df['line'].iloc[m]])
        df['colors'] = colorlist
        # print('colorlist',colorlist)

            
        # Create a folium map centered around the mean latitude and longitude
        map_center = [df['lat'].mean(), df['lon'].mean()]
        mymap = folium.Map(location=map_center, zoom_start=13, tiles='OpenStreetMap')
        
        
        # Add points to the map
        for index, row in df.iterrows():
            print(row)
            folium.Marker(location=[row['lat'], row['lon']], popup=row['line']+'\n'+row['label']+'('+row['device']+')\nlat/lon='+str(row['lat'])+','+str(row['lon'])+'\nts='+str(row['alarm_ts']), icon=folium.Icon(color=row['colors'])).add_to(mymap)

        # Save the map as an HTML file or display it in Jupyter Notebook
        mymap.save("map"+str(i)+".html")
        i+=1
        
            


device                       102.408.000213
label                                ΜΜ-218
line                   3-22 Κ/Δ ΜΑΡΚΟΠΟΥΛΟΥ
alarm_duration                      Unknown
lat                               37.867379
lon                               24.037221
alarm_ts          2023-12-16 00:36:00+02:00
colors                                  red
Name: 2023-12-16 00:36:00+02:00, dtype: object
device                       102.408.000214
label                                ΜΜ-197
line                   3-22 Κ/Δ ΜΑΡΚΟΠΟΥΛΟΥ
alarm_duration                      Unknown
lat                               37.862493
lon                               24.034781
alarm_ts          2023-12-16 00:36:00+02:00
colors                                  red
Name: 2023-12-16 00:36:00+02:00, dtype: object
device                       102.408.000228
label                                ΜΓ-019
line                      Ρ-230 ΝΕΑΣ ΜΑΚΡΗΣ
alarm_duration                        711.0
lat                       